<a href="https://colab.research.google.com/github/Fredrick-Li/Video-Stabalization/blob/main/AverageFilter_Video_Stablization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from google.colab import files
from google.colab.patches import cv2_imshow


In [2]:
#video = files.upload()


In [3]:
CAP = cv2.VideoCapture("videoplayback.mp4")
frame_number = int(CAP.get(cv2.CAP_PROP_FRAME_COUNT))


w = int(CAP.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(CAP.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = CAP.get(cv2.CAP_PROP_FPS)
codec = cv2.VideoWriter_fourcc('F','M', 'P', '4')
output = cv2.VideoWriter('video_out.avi', codec,fps, (2 * w, h))
_,prev = CAP.read();
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

In [ ]:

transformArray = np.zeros((frame_number-1, 3), np.float32)

for i in range(frame_number-2):
    prev_feature = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200,qualityLevel=0.01, minDistance=30, blockSize=3)
    successful, curr = CAP.read()

    if not successful:
      break

    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    curr_feature, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_feature, None)
    # assert prev_feature.shape == curr_feature.shape
    index = np.where(status==1)[0]
    prev_feature = prev_feature[index]
    curr_feature = curr_feature[index]

    # transform matrix
    matrix,_ = cv2.estimateAffine2D(prev_feature, curr_feature) ## problematic
    dx = matrix[0, 2]
    dy = matrix[1, 2]
    rotation  =np.arctan2(matrix[0, 1], matrix[0, 0])
    transformArray[i] = [dx, dy, rotation]
    prev_gray = curr_gray

    print("Frame: " + str(i) +  "/" + str(frame_number-2) + " -  Tracked points : " + str(len(prev_feature)))







In [5]:
def averageFilter(curve, length):
  window_size = 2*length +1
  filter = np.ones(window_size)/window_size
  padding = np.lib.pad(curve, (length, length), 'edge')
  averaged_curve = np.convolve(padding, filter, mode = 'same')
  averaged_curve = averaged_curve[length: -length]

  return averaged_curve

In [6]:
def smooth(trajectory):
  res = np.copy(trajectory)

  for i in range(3):
    res[:,i] = averageFilter(res[:,i], 50)

  return res


In [7]:
def boarderScale(frame):

  M =cv2.getRotationMatrix2D((frame.shape[1]/2, frame.shape[0]/2), 0, 1.05) # scale by 5%
  frame = cv2.warpAffine(frame, M, (frame.shape[1], frame.shape[0]))

  return frame


In [8]:
trajectory = np.cumsum(transformArray, axis=0)
trajectory.shape
smoothed_trajectory = smooth(trajectory)

noise = smoothed_trajectory-trajectory
smooth_transformArray= transformArray + noise

In [9]:
# reset to first frame
CAP.set(cv2.CAP_PROP_POS_FRAMES, 0)

for i in range(frame_number-1):
  successful, curr = CAP.read()
  if not successful:
    break

  dx = smooth_transformArray[i, 0]
  dy = smooth_transformArray[i, 1]
  rotation = smooth_transformArray[i, 2]

  M = np.zeros((2, 3), np.float32)

  M[0, 0]= np.cos(rotation)
  M[0, 1] = -np.sin(rotation)
  M[0, 2] = dx
  M[1, 0] = np.sin(rotation)
  M[1, 1] = np.cos(rotation)
  M[1, 2] = dy

  stabalized_frame = cv2.warpAffine(curr, M, (w, h))
  output_frame = boarderScale(stabalized_frame)
  output_frame = cv2.hconcat([curr, output_frame])

  if output_frame.shape[1] > 1.5*curr.shape[1]:
    cv2.resize(output_frame, (int(output_frame.shape[1]/2), int(output_frame.shape[0]/2)))
  

  cv2.waitKey(40)
  output.write(output_frame)

CAP.release()
output.release()
cv2.destroyAllWindows()



